In [2]:
import sys
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics.pairwise import linear_kernel

In [3]:
df = pd.read_csv('/home/ifte-home/Documents/mental_health/suicide/Suicide_Detection_processed.csv', index_col=0)

In [4]:
df = df[df['label']=='non-suicide']
df = df.dropna()
df = df.astype('str')
df = df[['text', 'label']]
df.reset_index(inplace=True, drop=True)

In [4]:
df

,text,label
0,weird affected compliment coming someone know ...,non-suicide
1,finally 2020 almost never hear 2020 year ever ...,non-suicide
2,everyone want edgy making self conscious feel ...,non-suicide
3,fuck verizon smart family even watch porn priv...,non-suicide
4,well screwed locked school toilet edit lived t...,non-suicide
...,...,...
115889,something today went sledding friend seem like...,non-suicide
115890,like rock going anything http musictaste space...,non-suicide
115891,tell many friend lonely everything deprived bo...,non-suicide
115892,probably taste like salty someone drank confirm,non-suicide


In [5]:
from ipynb.fs.full.preprocess_and_generate_dataset import generate_dataframe

In [6]:
class_dataframe = generate_dataframe(10)
class_dataframe = class_dataframe.rename(columns={'Post': 'text', 'Label':'label'})
CSSR_CAT = ['Indicator', 'Attempt','Behavior','Ideation', 'Supportive']
output = dict.fromkeys(CSSR_CAT)

In [10]:
best_df = pd.DataFrame()
for i in CSSR_CAT:
    classifier_df = class_dataframe[class_dataframe['label']==i]
    classifier_df.reset_index(inplace=True, drop=True)
#     dr_index = np.array(list(range(len(classifier_df))))
#     print('Length:', len(classifier_df), i)

    classifier_df = ' '.join(classifier_df['text'].values.tolist())
    main_df = pd.concat([pd.DataFrame([classifier_df], columns=['text']), df])
    main_df.reset_index(inplace=True, drop=True)
    print('Training')
    term_matrix, pipe = train_dataset(main_df)
#     compare_text = pipe.transform(classifier_df)
    temp, best_indices = collect_result(term_matrix)
    output[i] = temp
#     print(main_df.iloc[best_indices])
    
#     break
    new_df = main_df.iloc[best_indices]
    new_df['label'] = np.where(new_df['label'] == 'non-suicide', i, 'non-suicide')
    best_df = pd.concat([best_df, new_df])

output

Training


/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Training
Training
Training
Training


{'Indicator': 0.23291077928902698,
 'Attempt': 0.27430460288882347,
 'Behavior': 0.284891761030822,
 'Ideation': 0.27006992777845823,
 'Supportive': 0.25119659352840285}

In [11]:
output

{'Indicator': 0.23291077928902698,
 'Attempt': 0.27430460288882347,
 'Behavior': 0.284891761030822,
 'Ideation': 0.27006992777845823,
 'Supportive': 0.25119659352840285}

In [12]:
best_df

,text,label
69567,time read format wanted make book could done j...,Indicator
5571,depressed year story hope help raise awareness...,Indicator
34978,long text friend doubt anyone read tldr highly...,Indicator
28886,fuck idek name call badger year canadian dont ...,Indicator
87420,pickle pickle existence girl matter born write...,Indicator
...,...,...
41618,hentai helped become simp warning loooong text...,Supportive
31865,anyone need reason live would miss worth alive...,Supportive
14780,stressing life frankly sick tired world someti...,Supportive
40249,last advice offer make mistake often severe ot...,Supportive


In [13]:
best_df.to_csv('/home/ifte-home/Documents/mental_health/suicide/CSSRS/TFIDF_result_suicide_intensity.csv')

In [9]:
def train_dataset(frame):
    pipeline = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2), min_df=5, 
                                                  max_features=30000)), 
                         ('tfidf', TfidfTransformer(norm='l2'))])
    doc_term_matrix = pipeline.fit_transform(frame['text'])
    return doc_term_matrix, pipeline

def vectorize_and_get_avg_simiarity(doc_term_matrix):
    rank = linear_kernel(doc_term_matrix[0:1], doc_term_matrix)
    cosine_similarities = rank.flatten()
    related_docs = cosine_similarities.argsort()[:-500:-1]    
#     print(collect, related_docs)
    output = cosine_similarities[related_docs]
    avg_result = np.sum(output)/related_docs.size
    return avg_result, related_docs[1:101]

def collect_result(doc_term_matrix):
#     index_collection = []
#     collector = []
    
#     for i in range(len(suicide_df)):
    
    
    avg_result, best_index = vectorize_and_get_avg_simiarity(doc_term_matrix)
#     collector.append(out)
#     index_collection.extend(best_index)
        
#     best_ids = []
#     threshold=100
#     for i in Counter(index_collection).most_common():
#         best_ids.append(i[0])
#         threshold-=1
#         if threshold < 1:
#             break
                  
#     return sum(collector)/len(collector), best_ids
    return avg_result, best_index

In [58]:
# list of strings
lst = ['Geeks', 'For', 'Geeks', 'is', 
            'portal', 'for', 'Geeks']
 
# Calling DataFrame constructor on list
df = pd.DataFrame(lst)
print(df)

        0
0   Geeks
1     For
2   Geeks
3      is
4  portal
5     for
6   Geeks


In [67]:
pd.DataFrame(['Geeks For Geeks is portal for Geeks'], columns=['text'])

,text
0,Geeks For Geeks is portal for Geeks


a
b
d
c
e


In [139]:
term_matrix, pipe = train_dataset(main_df)

In [140]:
collect_result(classifier_df, term_matrix)

/home/ifte-home/venvs/exp-Env/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


nan